<a href="https://colab.research.google.com/github/Arshpreet-Singh-1/ai-machine-learning-for-coders/blob/main/3_4_Rock_Paper_Scissor_Multiclass_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Imports

In [1]:
import tensorflow as tf
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from keras.preprocessing import image

# 2. Training and Validation Dataset

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical'
)

--2022-06-18 07:45:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.128, 173.194.192.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   264MB/s    in 0.7s    

2022-06-18 07:45:19 (264 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

Found 2520 images belonging to 3 classes.


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip
    
local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical'
)

--2022-06-18 07:45:21--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 142.250.152.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/tmp/rps-test-set.zip’

/tmp/rps-test-set.z 100%[===================>]  28.15M   142MB/s    in 0.2s    

2022-06-18 07:45:21 (142 MB/s) - ‘/tmp/rps-test-set.zip’ saved [29516758/29516758]

Found 372 images belonging to 3 classes.


# 3. Build and Compile Model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [5]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# 4. Fit Model

In [6]:
model.fit(train_generator, epochs=25, validation_data = validation_generator, verbose = 1)

Epoch 1/25
79/79 [==============================] - 36s 284ms/step - loss: 1.2064 - accuracy: 0.3663 - val_loss: 0.9893 - val_accuracy: 0.5941
Epoch 2/25
79/79 [==============================] - 23s 293ms/step - loss: 0.8937 - accuracy: 0.6123 - val_loss: 0.4138 - val_accuracy: 0.9328
Epoch 3/25
79/79 [==============================] - 22s 275ms/step - loss: 0.5513 - accuracy: 0.7718 - val_loss: 0.1443 - val_accuracy: 0.9758
Epoch 4/25
79/79 [==============================] - 22s 273ms/step - loss: 0.3783 - accuracy: 0.8619 - val_loss: 0.0621 - val_accuracy: 0.9919
Epoch 5/25
79/79 [==============================] - 22s 274ms/step - loss: 0.2446 - accuracy: 0.9107 - val_loss: 0.1011 - val_accuracy: 0.9758
Epoch 6/25
79/79 [==============================] - 22s 277ms/step - loss: 0.1784 - accuracy: 0.9325 - val_loss: 0.0159 - val_accuracy: 1.0000
Epoch 7/25
79/79 [==============================] - 22s 275ms/step - loss: 0.1576 - accuracy: 0.9520 - val_loss: 0.7339 - val_accuracy: 0.7097

# 5. Running the Model

In [8]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)